### Two Body Simulation

A worksheet to simulate two spiraling black holes, using Post-Minkowski corrections to accounts for effects of relativity.

In [1]:
######
### This is the inital cell which prepares all of the necessary 
### packages and functions for use.
######

using Pkg
using DifferentialEquations
using Plots
using LSODA

#This is a homemade package containing the differential equations used to calculate the Hamiltonian numerically.
Pkg.activate("HamiltonianFunctions")
using HamiltonianFunctions

@userplot TwoBodyPlot
@recipe function f(tb::TwoBodyPlot)
    m, x, y, xarray, yarray, negx, posx, negy, posy = tb.args
    n = 100
    xlims --> (negx, posx)
    ylims --> (negy, posy)
    append!(xarray, x)
    append!(yarray, y)
    if size(xarray)[1] > n
        deleteat!(xarray, 1)
        deleteat!(yarray, 1)
    end
    linewidth --> range(0, m * 5, length = n)
    #seriesalpha --> range(0, 1, length = n)
    aspect_ratio --> 1
    label --> false
    xarray, yarray
end

 Activating environment at `~/Research/Post-Minkowski/notebooks/HamiltonianFunctions/Project.toml`


In [8]:
######
### This cell is for setting the parameters of the code. 
### These parameters are manually changed by the user as desired.
######

CSI = 3.00e8;
GSI = 6.647e-11;
MSUN = 1.989e30;

m1 = 1; # Mass of the first body in solar masses
m2 = 1; # Mass of the second body
D = 1000; # Initial separation of the bodies, in kilometers
ecc = 0.0; # eccentricity of the orbits
G = 1 / (16 * pi); # Gravitational constant
C = 1; #Speed of light
relativity = true; # A boolean value; if true, the code will apply Post-Minkowski relativistic corrections.
tspan = (0.0, 36050.0 * 40 * 10000); # The amount of time for which the simulation runs

M = MSUN
L = (GSI / G) * M * (C / CSI)^2
T = L * (C / CSI)

pUnits = M * L / T
KEUnits = pUnits * L / T;

In [ ]:
######
### This cell is where the code does most of its calculations. 
### If desired, a different ODE solver can also be chosen in this cell; 
### all that aren't currently being used are commented out. It will
### print out the changes in kinetic energy throughout the simulation in joules,
### and after each of those the ending kinetic energy divided by the initial,
### and then the distance between the bodies at the end of the simulation.
######

#Calculates factors based on eccentricity

if !(0 <= ecc <= 1)
    ecc = 0.0
end
qfac = 1 - ecc;
pfac = sqrt((1 + ecc)/(1 - ecc));

#Calculates inital position and momentum

totalM = m1 + m2
mu = m1 * m2 / (totalM)
if relativity
    #This is the Schwarzschild solution for calculating initial momentum. 
    #Keep in mind this will be imaginary if they get too close, 
    #and consider that in first order, the PM solution has repulsive force if they get too close.
    p_theta = mu * D * sqrt((G * totalM)/(D - (3 * G * totalM)))
else
    p_theta = sqrt(G * m1 * m2 * mu * D )
end

x2 = D / (1 + (m1 / m2));
x1 = x2 - D;
py2 = p_theta / D;
println("Initial momentum is ", py2)
println()
py1 = -py2;

#Set up initial arrays for ODE problem

c0 = [m1, m2, G]
q01 = [x1 * qfac, 0.0]
p01 = [0.0, py1 * pfac]
q02 = [x2 * qfac, 0.0]
p02 = [0.0, py2 * pfac]
h01 = [0]
q0 = collect(Base.Iterators.flatten([q01, q02]))
p0 = collect(Base.Iterators.flatten([p01, p02]))
u0 = collect(Base.Iterators.flatten([q01, p01, q02, p02, h01]));

pi1, pi2 = sqrt((p01[1]^2) + (p01[2]^2)), sqrt((p02[1]^2) + (p02[2]^2));
KEi1, KEi2 = pi1^2 / (2 * m1), pi2^2 / (2 * m2);
initialKE1, initialKE2 = KEi1 * KEUnits, KEi2 * KEUnits

#Create and solve ODE problem, finding ending data

if relativity
    #prob = ODEProblem(HamiltonianFunctions.PM, u0, tspan, c0)
    prob = HamiltonianProblem(HamiltonianFunctions.H, q0, p0, tspan, c0)
else
    #prob = ODEProblem(HamiltonianFunctions.Hamiltonian, u0, tspan, c0)
    prob = HamiltonianProblem(HamiltonianFunctions.HNewtonian, q0, p0, tspan, c0)
end

#sol = solve(prob, KahanLi8(), dt=0.1, reltol = 1.0e-8, abstol = 1.0e-8, saveat = 100);
#sol = solve(prob, RK4(), adaptive = true, reltol = 1.0e-8, abstol = 1.0e-8, saveat = 100);
#sol = solve(prob, Rosenbrock23(), reltol = 1.0e-2, abstol = 1.0e-2, saveat = 100);
#sol = solve(prob, Rodas5(), reltol = 1.0e-8, abstol = 1.0e-8, saveat = 100);
#sol = solve(prob, lsoda(), reltol = 1.0e-9, abstol = 1.0e-9, saveat = 100);
#sol = solve(prob, Tsit5(), reltol = 1.0e-8, abstol = 1.0e-8, saveat = 1000);
#sol = solve(prob, ImplicitEuler(), reltol = 1.0e-5, abstol = 1.0e-5, saveat = 100);
sol = solve(prob, SymplecticEuler(), reltol = 1.0e-9, abstol = 1.0e-9, saveat = 100, dt = 0.1);
#sol = solve(prob, ERKN4(), reltol = 1.0e-8, abstol = 1.0e-8, saveat = 100);

ss1 = sol[1,:]
ss2 = sol[2,:]
ss3 = sol[3,:]
ss4 = sol[4,:];
#ss3 = sol[5,:]
#ss4 = sol[6,:];

#Set an array that contains the distances between both bodies
xdis, ydis = ss1 - ss3, ss2 - ss4
distArr = (xdis.^2 + ydis.^2).^0.5

len = size(sol[1,:])[1]

pf1, pf2 = sqrt((sol[5,:][len]^2) + (sol[6,:][len]^2)), sqrt((sol[7,:][len]^2) + (sol[8,:][len]^2));
KEf1, KEf2 = pf1^2 / (2 * m1), pf2^2 / (2 * m2);
finalKE1, finalKE2 = KEf1 * KEUnits, KEf2 * KEUnits 
dKE1, dKE2 = finalKE1 - initialKE1, finalKE2 - initialKE2
xf, yf = ss1[len] - ss3[len], ss2[len] - ss4[len]
Df = sqrt(xf^2 + yf^2)
println("Change in KE of body 1 ", dKE1)
println("Ending KE of body 1 as a fraction of initial KE of body 1 ", KEf1/KEi1)
println()
println("Change in KE of body 2 ", dKE2)
println("Ending KE of body 2 as a fraction of initial KE of body 2 ", KEf2/KEi2)
println()
println("Final distance between objects ", Df, " = ", Df * L, " meters.")

#Create an array that's just the value of the Hamiltonian
#hamilArr = sol[9,:];
#deleteat!(hamilArr, 1);

Initial momentum is 0.0031541039048534154



In [10]:
######
### This cell creates plots that will be saved to the directory.
######

#Plot the Hamiltonian for error checking; this shouldn't change by much

#scatter(hamilArr)
#savefig("hamiltonian.png")

#Plot the change in distance between the bodies over time

plot(distArr)
savefig("distance.png")

#Plot the orbits of the bodies

plot(ss1, ss2, aspect_ratio = :equal)
plt = plot!(ss3, ss4, aspect_ratio = :equal)
savefig("orbits.png")

In [54]:
######
### If desired, this cell will animate the orbits of the bodies, 
### though the x and y bounds must be manually decided and inputted. 
######

xarr1, yarr1, xarr2, yarr2 = [], [], [], []
xneg, xpos, yneg, ypos = -20, 20, -15, 15

r = m1 / m2
m10 = r / (1 + r)
m20 = 1 - m10

anim = @gif for i = 1:size(ss1)[1]
    twobodyplot(m10, ss1[i], ss2[i], xarr1, yarr1, xneg, xpos, yneg, ypos)
    twobodyplot!(m20, ss3[i], ss4[i], xarr2, yarr2, xneg, xpos, yneg, ypos)
    end every 5

ProcessFailedException: failed process: Process(`/Users/zackarywindham/.julia/artifacts/dbb254dca678145c7d79410e96da494cc94f8a13/bin/ffmpeg -v 16 -i /var/folders/mk/kcwccfy96714742ht9qvmbx80000gn/T/jl_neKcbM/%06d.png -vf palettegen=stats_mode=diff -y /var/folders/mk/kcwccfy96714742ht9qvmbx80000gn/T/jl_neKcbM/palette.bmp`, ProcessExited(255)) [255]
